# Comments classification: offensive/non-offensive

1. Import Dataset

In [1]:
import pandas as pd
df = pd.read_csv('Dataset/CVMAD_04.csv', encoding='UTF-8')### if it doesn't work try 'utf-8-sig'
df.head()

,content,decision
0,یخ علی الخماح مخسر للسیده وجها مازال کتهضرشوهت...,1
1,یاک مطلقها علاش کیحاسبها داب,1
2,یاخوفي من بدعة التفاخر بالقتل والتشرمیل والمعص...,1
3,يوم يكون الصوت سعره يتحدى 30درهم هنا سيصبح الت...,0
4,يوسف زروالي هو لقتقصدك لدخلو معاه جميعة شباب ا...,0


2. Data Encoding

    Word Embedding

In [4]:
import numpy as np
word_embeddings = {}
f = open('ma_model_cbow_mix.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()
print('Total %s word vectors in MA CBOW model 300d.' % len(word_embeddings))

Total 82906 word vectors in MA CBOW model 300d.


In [5]:
def get_sentence_vectors(sentence):
    sentence_vector = np.zeros((300,))
    if len(sentence)==0:
        return sentence_vector
    for word in sentence:
        if word in word_embeddings:
            sentence_vector += word_embeddings[word]
    sentence_vector = sentence_vector/len(sentence)
    return sentence_vector

In [8]:
len(word_embeddings)

82906

In [6]:
df["content"] = df["content"].apply(get_sentence_vectors)
df.head()

,content,decision
0,"[0.03822386347585254, 0.030719361233490484, -0...",1
1,"[-0.016081052499690225, -0.011788641102612019,...",1
2,"[0.014637194904140555, 0.01521033041895582, -0...",1
3,"[0.027888019090971432, 0.02286620584929043, -0...",0
4,"[0.04122190000306742, 0.028468514847404817, -0...",0


3. Data splitting

In [7]:
X = df.content
y = df.decision

In [8]:
""" Scaling and Splitting the dataset into training, validation and test sets """
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#scaler = StandardScaler()
#X_sacaled=scaler.fit_transform(X)

train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=50, test_size=0.3)

In [9]:
train_X=np.array(train_X.to_list())

In [10]:
test_X=np.array(test_X.to_list())

In [61]:
train_X.shape

(16100, 300)

4. Modeling

In [26]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=200, warm_start=True)

In [27]:
clf.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [28]:
pred_y=clf.predict(test_X)

In [29]:
# import the metrics class
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(test_y, pred_y)
cnf_matrix

array([[2051, 1374],
       [1386, 2089]], dtype=int64)

In [30]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

           0       0.60      0.60      0.60      3425
           1       0.60      0.60      0.60      3475

   micro avg       0.60      0.60      0.60      6900
   macro avg       0.60      0.60      0.60      6900
weighted avg       0.60      0.60      0.60      6900

